# Машинное обучение, ФКН ВШЭ

## Практическое задание 14. Оптимизация гиперпараметров.

### Общая информация

Дата выдачи: 04.06.2022

Мягкий дедлайн: 19.06.2022 23:59 MSK

Жёсткий дедлайн: 21.06.2022 23:59 MSK

### Оценивание и штрафы

Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 15 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

### Формат сдачи

Задания сдаются через систему anytask. Посылка должна содержать:

- Ноутбук `homework-practice-14-Username.ipynb`
- Модули `distributions.py` и `optimizers.py`, содержащие написанный вами код

Username — ваша фамилия на латинице

### О задании

В этом задании вам предстоит разработать свой собственный AutoML фреймворк, который будет подбирать оптимальные значения гиперпараметров для определенной модели и обучающих данных. Попутно вы познакомитесь с основными концепциями Байесовской оптимизации и узнаете, как Гауссовские процессы и метод Парзеновского окна применяются на практике. Если вы все сделаете правильно, то на выходе получится фреймворк, не уступающий популярной `optuna`.

## 1. AutoML фреймворк (7.5 баллов)

Наш фреймворк будет работать по аналогии с классом [`OptunaSearchCV`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.integration.OptunaSearchCV.html#optuna.integration.OptunaSearchCV) из пакета `optuna`, который интегрирован с интерфейсом `sklearn`. Общая схема следующая: сначала мы выбираем модель, для которой будем подбирать гиперпараметры, и задаем распределения на них (решаем, будет ли параметр вещественным или целочисленным; определяем границы; выбираем линейную или логарифмическую шкалу). Далее, на каждой итерации мы выбираем новый набор значений гиперпараметров и оцениваем качество модели на них с помощью кросс-валидации. За ограниченное число итераций находим лучший набор гиперпараметров и обучаем итоговую модель на всех данных. Условимся, что мы хотим **максимизировать** значение метрики (в случае, если мы хотим минимизировать метрику, будем брать ее с минусом). Отличие между различными стратегиями поиска заключается в том, как именно выбираются новые значения гиперпараметров. Подробнее об этом ниже.

Вашу реализацию кода нужно будет протестировать через [ЯндексКонтест](https://contest.yandex.ru/contest/37803/). Помимо стандартных библиотек Python 3.7, в контесте доступны `numpy=1.21.6`, `scipy=1.7.3` и `scikit-learn=1.0.2`, а также их зависимости.

**Задание 1.1 (0.5 балла).** Реализуйте распределения гиперпараметров в файле `distributions.py`. Каждый класс должен поддерживать метод `sample`, который возвращает выборку из распределения. Кроме того, для численных распределений нам понадобится метод `scale`, который переводит выборку в промежуток $[0, 1]$ (с линейной шкалой) (за подробностями обращайтесь к док-строкам в базовых классах). Чуть позже станет понятнее, как нам пригодится эта функция. Задача в контесте: [ссылка](https://contest.yandex.ru/contest/37803/problems/A/). \
**>>>Done**

**Задание 1.2 (1 балл).** Реализуйте методы `sample_params`, `cross_validate`, `fit`, `predict` и `predict_proba` класса `BaseOptimizer` в файле `optimizers.py`. Обратите внимание, что разбиение на фолды должно быть одинаковым для каждой итерации оптимизации (чтобы оценивать наборы гиперпараметров честно). Кроме того, не забывайте о воспроизводимости экспериментов: запуски всех алгоритмов, содержащих случайность, должны контролироваться с помощью `random_seed`. Также сохраняйте историю параметров и метрики в поля `self.params_history` и `self.scores_history`.\
**>>>Done**

**Задание 1.3 (0.5 балла).** Реализуйте простейшую возможную стратегию &mdash; стратегию случайного поиска в классе `RandomSearchOptimizer` из файла `optimizers.py`. На каждой новой итерации алгоритма мы генерируем новый случайный набор гиперпараметров, не производя никакого дополнительного отбора. Задача в контесте: [ссылка](https://contest.yandex.ru/contest/37803/problems/B/)\
**>>>Done**

In [6]:
import numpy as np
from distributions import UniformDistribution, CategoricalDistribution

In [3]:
UniformDistribution(0, 10).sample(5)

array([4.88160636, 2.49717338, 5.51420902, 6.96908028, 3.98782348])

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, KFold, StratifiedKFold
from sklearn import datasets

X, y = datasets.load_breast_cancer(return_X_y=True)
X.shape, y.shape

params = {"penalty":"l2", "solver":"saga", "C":0.1, "class_weight":"auto", "max_iter":5000}
clf = LogisticRegression()
clf.set_params(**params)

LogisticRegression(C=0.1, class_weight='auto', max_iter=5000, solver='saga')

In [5]:
cross_validate(clf, X, y, cv=5, scoring=None)['test_score'].mean()

0.9226672876882471

### Байесовская оптимизация

Далее мы перейдем к более продвинутым методам поиска гиперпараметров. Одним из наиболее распространенных подходов к этой задаче является **Байейсовская оптимизация**. Ее суть состоит в следующем: на каждой итерации мы будем аппроксимировать распределение $p(y|x)$, где $y$ &mdash; это значение метрики, а $x$ &mdash; значение набора гиперпараметров. Допустим, что $y^*$ &mdash; это наилучшее (наибольшее) текущее значение метрики. Чтобы выбрать новое значение гиперпараметров, для которого мы будем учить модель, нам понадобится величина под названием **Expected Impovement (EI)**. Она определяется следующим образом:

$$
\text{EI}_{y^*}(x) = \mathbb{E}_{y \sim p(y|x)} \Big[\max(0, y - y^*)\Big] = \int \max(0, y - y^*) p(y|x) dy
$$

Соответственно, чем выше значение EI, тем большего увеличения $y^*$ мы ожидаем, обучив модель с гиперпараметрами $x$. Таким образом, на каждой итерации оптимизации мы будем семлировать несколько кандидатов $x_i, i=1, \dots, N$ из исходного распределения гиперпараметров и выбирать из них наилучшего путем максимизации EI: $x_{\text{new}} = \arg\max_{x_i} \big\{\text{EI}_{y^*} (x_i)\big\}$. Нам осталось обсудить, как именно аппроксимировать распределение $p(y|x)$.

### Гауссовские процессы

[Гауссовские процессы](https://en.wikipedia.org/wiki/Gaussian_process) (ГП) &mdash; одна из популярных опций для задания распределения $p(y|x)$. Если не вдаваться в глубокую теорию, то можно воспринимать ГП как случайные функции. ГП задаются двумя параметрами: функцией среднего $\mu(x)$ и ковариационной функцией $K(x, x')$. Процесс называется гауссовским, поскольку в каждой отдельно взятой точке $x$ мы имеем нормальное распределение: $y(x) \sim \mathcal{N}\big(\mu(x), K(x, x)\big)$. А условие $\text{Cov}\big(y(x), y(x')\big) = K(x, x')$ объясняет название ковариационной функции. Как правило, для практических применений достаточно так называемых стационарных ГП, которые отличают константная функция среднего $\mu(x)=\text{const}$ (чаще всего даже $\mu(x)=0$) и ковариационная функция, зависящая только от расстояния между точками: $K(x, x') = K\big(\|x-x'\|\big)$. Часто под переменной $x$ понимают время (отсюда название "процесс"), но мы будет рассматривать в качестве $x$ наше пространство гиперпараметров.

На первый взгляд может показаться, что ГП мало чем могут быть полезны: ведь в каждой точке $x$ у нас свое собственное случайное значение $y$. Но оказывается, что если ковариационная функция непрерывна в нуле, то (почти все) реализации ГП (то есть интересующие нас случайные значения $y(x)$) непрерывны. Таким образом, каждая реализация ГП соответствует некоторой непрерывной функции, а значит, ГП можно использовать для регрессии.

Если задана обучающая выборка $\{(x_i, y_i)\}_{i=1}^{\ell}$, то мы можем посчитать распределение $p(y|x) = \mathcal{N}\big(y\big|\mu(x), \sigma^2(x)\big)$ для любой точки $x$ (замечание: здесь уже не будет выполняться $\mu(x)=0$, так как распределение на $y$ апостериорное, то есть зависит от обучающих данных). При этом, для разных ковариационных функций мы будем получать разные распределения. Обычно в качестве ковариационных функций используют ядра (да-да, те самые из теоремы Мерсера). Нам повезло, что все это уже реализовано в классе [`sklearn.gaussian_process.GaussianProcessRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html#sklearn.gaussian_process.GaussianProcessRegressor), и мы можем сконцентрироваться на алгоритмах оптимизации гиперпараметров.

Ниже приведен пример регрессии с помощью ГП:
![](https://www.researchgate.net/profile/Florent-Leclercq/publication/327613136/figure/fig1/AS:749406701776896@1555683889137/Illustration-of-Gaussian-process-regression-in-one-dimension-for-the-target-test.png)

Впрочем, в нашем случае, картинка будет выглядеть скорее так, поскольку $x$ будет многомерным:
![](https://raw.githubusercontent.com/jmaronas/TGP.pytorch/master/images/2D_GP.gif)

**Задание 1.4 (0.75 балла)**. Выведите аналитическую формулу для подсчета EI в случае $p(y|x) = \mathcal{N}\big(y\big|\mu(x), \sigma^2(x)\big)$ (подсказка: вам может пригодится значение функции распределения нормальной с.в.). \
Обозначим за $\phi(x) = \dfrac{1}{\sqrt{2\pi}}e^{-x^2 / 2}$ - плотность стандартной нормальной с.в. \
Стандартизируем значения метрики и обозначим их следующим образом:
$$
u^* = \dfrac{y^* - \mu}{\sigma}, \quad u = \dfrac{y - \mu}{\sigma} \text{, where } u \sim \phi(u).
$$
Тогда запишем expected improvement с новой плотностью:
$$
\text{EI}_{y^*}(x) = \int \max(0, y - y^*) p(y|x) dy = \\
= \int_{\mathbb{R}} (y - y^*) I\{y > y^*\} p(y|x) dy = \\
= \int_{\mathbb{R}} (\mu + \sigma u - y^*) I\{u > u^*\} \phi(u) du = \\
= \int_{u^*}^{+\infty} (\mu - y^* + \sigma u) \phi(u) du = \\
= (\mu - y^*)\, \big(1 - \Phi(u^*)\big) + \dfrac{\sigma}{\sqrt{2\pi}} \int_{u^*}^{+\infty}  u e^{-u^2 / 2} du = \\
= (\mu - y^*)\, \big(1 - \Phi(u^*)\big) - \dfrac{\sigma}{\sqrt{2\pi}} e^{-u^2 / 2} \bigg|_{u^*}^{+\infty} = \\
= (\mu - y^*)\, \big(1 - \Phi(u^*)\big) - \sigma \big(0 - \phi(u^*)\big) = \\
= - \sigma u^* \, \big(1 - \Phi(u^*)\big) + \sigma \phi(u^*).
$$

In [7]:
from scipy.stats import norm
mu = np.arange(5)
sigma = np.arange(1, 6)
y_star = 5

u_star = (y_star - mu) / sigma
sigma * (u_star * norm.cdf(y_star - mu) + norm.pdf(u_star))

array([5.00000005, 4.10785525, 3.72186248, 3.36276104, 2.79655822])

In [8]:
X_num = np.zeros([100, 3])
for i in range(3):
    sc = np.arange(100)
    X_num[:, i] = sc
np.argmax(sc)

99

In [9]:
X.shape, y.shape

((569, 30), (569,))

In [10]:
import warnings 
warnings.filterwarnings('ignore')

In [11]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, WhiteKernel, RBF

kernel = WhiteKernel() + RBF()
gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(X, y)

In [12]:
mu, sigma = gpr.predict(np.ones([250, 30]), return_std=True)

In [13]:
mu.shape, sigma.shape

((250,), (250,))

In [14]:
u_star = (5 - mu) / sigma
ei = - sigma * u_star * (1 - norm.cdf(u_star)) + sigma * norm.pdf(u_star)
ei.shape

(250,)

In [29]:
d = {'a': 111, 'b': 222, 'c': 333}
for i, p in enumerate(d.keys()):
    print(p)

a
b
c


In [40]:
d[d.keys()[0]]

TypeError: 'dict_keys' object is not subscriptable

In [155]:
sx = np.arange(30).reshape(-1, 3)
np.argmax(sx, axis=0)

array([9, 9, 9], dtype=int64)

Теперь разберемся, как адаптировать ГП под наши нужды. Предлагается следующий план действий:

1. Пускай на очередной итерации алгоритма $j$ у нас есть выборка $\{(x_i, y_i)\}_{i=1}^{j-1}$, где $x_i$ &mdash; значение набора гиперпараметров, а $y_i$ &mdash; соответствующее ему значение метрики, посчитанное по кросс-валидации.


2. Если размер выборки меньше значения `self.num_dry_runs`, то возвращаем случайное значение гиперпараметров вместо запуска алгоритма. Таким образом, первые `self.num_dry_runs` итераций поиска подчиняются случайной стратегии, чтобы "набрать статистику" для работы алгоритма.


3. ГП работает только с непрерывными величинами, поэтому давайте переведем значения гиперпараметров в диапазон $[0, 1]$ (тут-то нам и понадобится функция `scale`) и получим отнормированные гиперпараметры $x_i \to \tilde{x}_i$. Как вы догадываетесь, такой трюк сработает только с численными гиперпараметрами (как с непрерывными, так и с дискретными). Что делать с категориальными гиперпараметрами, обсудим позже. 


4. Далее обучаем ГП на выборку $\{(\tilde{x}_i, y_i)\}_{i=1}^{j-1}$ (в качестве ковариационной функции возьмите сумму константного, белого и RBF ядер). 


5. Семплируем кандидатов в новые значения гиперпараметров $x^{\text{new}}_1, ..., x^{\text{new}}_N$ и преобразуем их функцией `scale`. Для каждого набора предсказываем параметры распределения $\mu(\tilde{x}), \sigma^2(\tilde{x})$, затем вычисляем $\text{EI}_{y^*}(\tilde{x})$ по формуле, посчитанной вами выше, и выбираем набор гиперпараметров $\tilde{x}^\text{new}_{*}$ с наибольшим значением EI.


6. Выбираем исходный набор гиперпараметров $x^\text{new}_{*}$, соответствующий $\tilde{x}^\text{new}_{*}$. Вуаля! Мы получили новый набор численных гиперпараметров, на котором будем валидировать модель.


Для категориальных гиперпараметров попробуем провернуть что-нибудь похожее. Пусть у нас есть выборка $\{(f_i, y_i)\}_{i=1}^{j-1}$, где $f$ &mdash; интересующий нас категориальный гиперпараметр, $f_i \in \{1, \dots, C\}$ &mdash; конечное множество категорий. Зафиксируем $c \in \{1, \dots, C\}$ и рассмотрим $p(y|с) = \mathcal{N}\big(y\big|\mu(с), \sigma^2(с)\big)$. Определим параметры распределений следующим образом (как среднее и сглаженную дисперсию по категории):

$$
\mu(c) = \begin{cases}
    \cfrac{\sum_i [f_i = c] \cdot y_i}{\sum_i [f_i = c]},& \sum_i [f_i = c] > 0 \\
    y^*,& \sum_i [f_i = c] = 0
\end{cases}
\quad\quad\quad
\sigma^2(c) = \frac{1 + \sum_i [f_i = c] (y_i - \mu(c))^2}{1 + \sum_i [f_i = c]}
$$

В итоге, как и в случае с численными гиперпараметрами, мы находим категорию с лучшей EI. Впрочем, обычно такие гиперпараметры имеют не очень много возможных значений (скажем, не больше 10 категорий, а в реальности скорее 2-3: например, критерий при построении дерева), поэтому мы можем себе позволить **перебрать все значения** категориального признака **вместо семплирования** (что и требуется от вас в реализации). Обратите внимание, что категориальные гиперпараметры нужно обрабатывать отдельно (то есть они не имеют ничего общего с ГП, который мы обучаем по численным гиперпараметрам). Конечно, у такого подхода есть значительный минус: категории рассматриваются независимо друг от друга и от численных гиперпараметров, но уж лучше так, чем никак. 

**Задание 1.5 (2 балла)**. Реализуйте недостающие методы в классе `GPOptimizer` в файле `optimizers.py`. Задачи в контесте: [раз](https://contest.yandex.ru/contest/37803/problems/С/), [два](https://contest.yandex.ru/contest/37803/problems/D/) \
**>>>Done**

### Tree-structured Parzen estimator (TPE)

Рассмотрим иной подход к приближению распределения $p(y|x)$. Он основан на использовании формулы Байеса:

$$
p(y|x) = \frac{p(x|y)p(y)}{p(x)}
$$

Эта формула позволяет нам моделировать распределение $p(x|y)$ вместо $p(y|x)$. Обозначим за $y^*$ $\gamma$-квантиль распределения $p(y)$, то есть $p(y < y^*)=\gamma$ (в отличие от ГП, где $y^*$ выступает как максимум распределения). Идея TPE заключается в следующем выражении для плотности $p(x|y)$:

$$
p(x|y) = \begin{cases}
    g(x),& y \ge y* \\
    l(x),& y < y^*,
\end{cases}
$$

где $l(x)$ &mdash; это аппроксимация распределения $x$ при значениях целевой метрики меньше порогового $y^*$, а $g(x)$ &mdash; наоборот, выше порогового. Оказывается, что такой способ задать распределение позволяет нам очень просто находить максимум EI:

**Задание 1.6 (0.75 балла)**. Докажите, что точка максимума EI совпадает с точкой максимума $\cfrac{g(x)}{l(x)}$.

$$
\text{EI}_{y^*}(x) = \int \max(0, y - y^*) p(y|x) dy = \\
\int_{y^*}^{+\infty} (y - y^*) p(y|x) dy = \\
\int_{y^*}^{+\infty} (y - y^*) \dfrac{p(x|y) p(y)}{p(x)} dy.
$$
Так как по построению $p(y < y^*) = \gamma$ и $p(x) = \int p(x|y) p(y) dy = \gamma l(x) + (1 - \gamma) g(x)$, то
$$
\int_{y^*}^{+\infty} (y - y^*) \dfrac{p(x|y) p(y)}{p(x)} dy = \\
l(x) \int_{y^*}^{+\infty} (y - y^*) p(y) dy = \\
\gamma y^* l(x) + l(x) \int_{y^*}^{+\infty} p(y) dy.
$$
Отсюда, Expected Improvement пропорционален:
$$
\text{EI}_{y^*}(x) = \dfrac{\gamma y^* l(x) + l(x) \int_{y^*}^{+\infty} p(y) dy}{\gamma l(x) + (1 - \gamma) g(x)} \propto \bigg( \gamma + \dfrac{g(x)}{l(x)} (1 - \gamma)\bigg)^{-1}.
$$

Источник: https://proceedings.neurips.cc/paper/2011/file/86e8f7ab32cfd12577bc2619bc635690-Paper.pdf


Таким образом, мы можем разделить все опробованные наборы гиперпараметров на две группы по значению целевой метрики, на каждой из них обучить свою оценку плотности ($g(x)$ или $l(x)$) и выбрать в качестве кандидата тот набор гиперпараметров, который имеет набольшее отношение плотностей. Такая процедура будет гарантировать нам максимизацию EI. В качестве алгоритма для приближения плотности мы будем пользоваться методом Парзеновского окна (он же Kernel Density Estimation, KDE), что и объясняет название оптимизатора (кстати, именно TPE находится под капотом у `optuna`). Как вы догадываетесь, KDE тоже реализован в [`sklearn.neighbors.KernelDensity`](https://www.youtube.com/watch?v=dQw4w9WgXcQ). И снова, такой подход позволяет обрабатывать только численные гиперпараметры. Получаем следующий алгоритм:

1. Пускай на очередной итерации алгоритма $j$ у нас есть выборка $\{(x_i, y_i)\}_{i=1}^{j-1}$, где $x_i$ &mdash; значение набора гиперпараметров, а $y_i$ &mdash; соответствующее ему значение метрики, посчитанное по кросс-валидации.


2. Если размер выборки меньше значения `self.num_dry_runs`, то возвращаем случайное значение гиперпараметров вместо запуска алгоритма.


3. Переводим значения гиперпараметров в диапазон $[0, 1]$ и получаем отнормированные гиперпараметры $x_i \to \tilde{x}_i$.


4. Считаем $\gamma$-квантиль выборки $y_1, \dots, y_{j-1}$ и делим значения гиперпараметров $\tilde{x}_1, \dots, \tilde{x}_{j-1}$ на две группы (воспользуйтесь `np.quantile`). 


5. На каждой группе обучаем KDE, чтобы получить оценки $g(\tilde{x})$ и $l(\tilde{x})$ (обратите внимание, что алгоритм возвращает логарифм оценки плотности). Используйте гауссовское ядро, в качестве параметра `bandwidth` возьмите медианное расстояние до 1-го ближайшего соседа в полной выборке. Остальные параметры оставьте по умолчанию.


6. Семплируем кандидатов в новые значения гиперпараметров $x^{\text{new}}_1, ..., x^{\text{new}}_N$ и преобразуем их функцией `scale`. Определяем точку:

$$
\tilde{x}^\text{new}_{*} = \arg\max_{i} \bigg\{\frac{g(\tilde{x}^{\text{new}}_i)}{l(\tilde{x}^{\text{new}}_i)}\bigg\}
$$


7. Выбираем исходный набор гиперпараметров $x^\text{new}_{*}$, соответствующий $\tilde{x}^\text{new}_{*}$.


Для категориальных гиперпараметров $f_i \in \{1, \dots, C\}$ нам тоже нужно определить $g(x)$ и $l(x)$. Определим вероятность категории $с \in \{1, \dots, C\}$ пропорциональной следующей величине:

$$
p(c) \propto N p_c + \sum_i [f_i = c], 
$$

где $N$ &mdash; число наборов гиперпараметров в группе (то есть это количество будет различным для $g(c)$ и $l(c)$), а $p_c$ &mdash; априорная вероятность категории (в нашем случае категории априорно равновероятны, $p_c = 1\,/\,C$). Сама вероятность $p(c)$ получается нормировкой по всем категориям гиперпараметра. И снова, мы будем **перебирать все значения** категориального признака **вместо семплирования**. Как и в случае ГП, категориальные гиперпараметры будут учитываться независимо от численных и друг от друга (тут даже можно выписать факторизацию распределения в виде $g(x) = g_{\text{num}}(x_\text{num}) \prod_{\text{cat}} g_{\text{cat}}(x_\text{cat})$, аналогично для $l(x)$, пусть эти формулы и не используются явно).

Выглядит итерация метода примерно так:

![](https://camo.githubusercontent.com/d14da729457267a66ebc9942b1ce57372adad287b74a1af5e0793f283c9bf1aa/687474703a2f2f6e657570792e636f6d2f5f696d616765732f7470652d73616d706c65642d63616e646964617465732e706e67)

**Задание 1.7 (2 балла)**. Реализуйте недостающие методы в классе `TPEOptimizer` в файле `optimizers.py`. Задачи в контесте: [раз](https://contest.yandex.ru/contest/37803/problems/E/), [два](https://contest.yandex.ru/contest/37803/problems/F/)
**>>>Done**

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import distributions as D
import optimizers as O

## 2. Эксперименты (7.5 баллов)

**Задание 2.1 (1 балл)**. Сгенерируйте случайную выборку наборов гиперпараметров и выборку значений метрик. Замерьте время работы метода `select_params` для `GPOptimizer` и `TPEOptimizer`. Постройте график зависимости времени работы от размера выборки, сделайте выводы. Какую алгоритмическую сложность имеют алгоритмы?

In [ ]:
# Эксперименты еще не доделал, кину после мягкого дедлайна

Дальнейшие эксперименты будет проводить на [наборе данных](https://www.kaggle.com/datasets/piyushagni5/white-wine-quality) о качестве белого вина.

**Задание 2.2 (3 балла)**. Будем тестировать оптимизаторы на (барабанная дробь...) бустингах. Выберите библиотеку градиентного бустинга, которая приходится вам по душе и задайте распределения на гиперпараметры. Среди ваших гиперпараметров обязательно должны быть число деревьев, глубина дерева и learning rate, также добавьте еще 4-5 гиперпараметров по вашему желанию (среди них должен быть хотя бы один категориальный). Подумайте, какие распределения лучше всего подходят для каждого гиперпараметра.

Как и всегда, разделите выборку на обучение и тест, предсказывать будем рейтинг вина (колонка `quality`). Хоть рейтинги и можно упорядочить, давайте решим задачу многоклассовой классификации на этих данных. В качестве метрики качества возьмем log loss. Мы будем сравнивать три стратегии для поиска, реализованные вами и [`OptunaSearchCV`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.integration.OptunaSearchCV.html#optuna.integration.OptunaSearchCV) из библиотеки `optuna`, который имеет такой же интерфейс. Давайте ограничим наш бюджет поиска 40 итерациями. Обучите каждый из этих четырех методов по нескольку раз, чтобы посмотреть на дисперсию метрики (как минимум 3 раза, а в идеале раз 5, если позволяет время).

In [61]:
# !pip install optuna

In [80]:
from optuna.integration import OptunaSearchCV
import optuna.distributions as opt_distr

In [107]:
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')


from distributions import CategoricalDistribution, UniformDistribution, IntUniformDistribution
from optimizers import GPOptimizer, TPEOptimizer, RandomSearchOptimizer

In [105]:
df = pd.read_csv('winequality-white.csv', sep=';')

X = df.drop('quality', axis=1)
y = df['quality']
y.value_counts()

6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: quality, dtype: int64

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=42, 
                                                    stratify=y)
est = LGBMClassifier(class_weight='balanced')

### Optuna Search

In [114]:
opt_params_distr = {'n_estimators': opt_distr.IntUniformDistribution(50, 300), 
                'max_depth': opt_distr.IntUniformDistribution(3, 6), 
                'learning_rate': opt_distr.UniformDistribution(0.01, 0.1), 
                'boosting_type': opt_distr.CategoricalDistribution(['gbdt', 'dart', 'goss']), 
                'num_leaves': opt_distr.IntUniformDistribution(31, 50), 
                'min_child_samples': opt_distr.IntUniformDistribution(20, 50), 
                'reg_alpha ': opt_distr.UniformDistribution(0.1, 10)
               }

In [123]:
optuna_search = OptunaSearchCV(est, opt_params_distr, cv=5, random_state=42, 
                              scoring='neg_log_loss')
optuna_search.fit(X_train, y_train)

[I 2022-06-19 23:22:12,746] A new study created in memory with name: no-name-5e16cf04-cd61-4233-b36a-97a69f7db8a8


[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 1.8445909353908974
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 1.8445909353908974
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 1.8445909353908974
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 1.8445909353908974


[I 2022-06-19 23:22:15,157] Trial 0 finished with value: -1.5695438859419177 and parameters: {'n_estimators': 73, 'max_depth': 6, 'learning_rate': 0.016214620678624378, 'boosting_type': 'dart', 'num_leaves': 35, 'min_child_samples': 25, 'reg_alpha ': 1.8445909353908974}. Best is trial 0 with value: -1.5695438859419177.


[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 9.74005282941547
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 9.74005282941547
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 9.74005282941547
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 9.74005282941547


[I 2022-06-19 23:22:15,847] Trial 1 finished with value: -1.1328189584033048 and parameters: {'n_estimators': 66, 'max_depth': 4, 'learning_rate': 0.09993257743060854, 'boosting_type': 'gbdt', 'num_leaves': 40, 'min_child_samples': 46, 'reg_alpha ': 9.74005282941547}. Best is trial 1 with value: -1.1328189584033048.


...

[I 2022-06-19 23:22:25,090] Trial 8 finished with value: -1.5492082557851585 and parameters: {'n_estimators': 58, 'max_depth': 3, 'learning_rate': 0.03956263629024348, 'boosting_type': 'dart', 'num_leaves': 36, 'min_child_samples': 28, 'reg_alpha ': 3.4494167388040005}. Best is trial 2 with value: -1.1225663124676362.


[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 3.4494167388040005
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 6.339030519832575
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 6.339030519832575
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 6.339030519832575
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 6.339030519832575
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 6.339030519832575


[I 2022-06-19 23:22:25,713] Trial 9 finished with value: -1.238178891940077 and parameters: {'n_estimators': 74, 'max_depth': 3, 'learning_rate': 0.0759169163961891, 'boosting_type': 'goss', 'num_leaves': 35, 'min_child_samples': 29, 'reg_alpha ': 6.339030519832575}. Best is trial 2 with value: -1.1225663124676362.


[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 8.295457145583939


OptunaSearchCV(estimator=LGBMClassifier(boosting_type='dart',
                                        class_weight='balanced',
                                        learning_rate=0.06619829018245553,
                                        max_depth=6, min_child_samples=21,
                                        n_estimators=490, num_leaves=48,
                                        random_state=42,
                                        reg_alpha =1.0658688513690675),
               param_distributions={'boosting_type': CategoricalDistribution(choices=('gbdt', 'dart', 'goss')),
                                    'learning_rate': UniformDistr...h=0.1, low=0.01),
                                    'max_depth': IntUniformDistribution(high=6, low=3, step=1),
                                    'min_child_samples': IntUniformDistribution(high=50, low=20, step=1),
                                    'n_estimators': IntUniformDistribution(high=100, low=50, step=1),
                   

In [117]:
optuna_search.best_params_
optuna

{'n_estimators': 66,
 'max_depth': 4,
 'learning_rate': 0.09993257743060854,
 'boosting_type': 'gbdt',
 'num_leaves': 40,
 'min_child_samples': 46,
 'reg_alpha ': 9.74005282941547}

### Random Search

In [98]:
params_distr = {'n_estimators': IntUniformDistribution(50, 300), 
                'max_depth': IntUniformDistribution(3, 6), 
                'learning_rate': UniformDistribution(0.01, 0.1), 
                'boosting_type': CategoricalDistribution(['gbdt', 'dart', 'goss']), 
                'num_leaves': IntUniformDistribution(20, 50), 
                'min_child_samples': IntUniformDistribution(20, 50), 
                'reg_alpha ': UniformDistribution(0.1, 10)
                }

In [110]:
%%time
r_search = RandomSearchOptimizer(est, params_distr, num_runs=40, random_state=42, 
                                 scoring=log_loss)
r_search.fit(X_train, y_train)

CPU times: total: 34min 21s
Wall time: 4min 42s


RandomSearchOptimizer(estimator=LGBMClassifier(boosting_type='goss',
                                               class_weight='balanced',
                                               learning_rate=0.09556428757689246,
                                               max_depth=6,
                                               min_child_samples=40,
                                               n_estimators=152, num_leaves=48,
                                               random_state=42,
                                               reg_alpha =1.6445845403801216),
                      num_runs=40,
                      param_distributions={'boosting_type': <distributions.CategoricalDistribution object at 0x000001E2DD08BAC0>...
                                           'min_child_samples': <distributions.IntUniformDistribution object at 0x000001E2DD08A7D0>,
                                           'n_estimators': <distributions.IntUniformDistribution object at 0x000001E2DD02A140

### GP Optimizer

In [111]:
%%time
gp_search = GPOptimizer(est, params_distr, num_runs=40, random_state=42, 
                        num_dry_runs=5, num_samples_per_run=10, 
                        scoring='neg_log_loss')
gp_search.fit(X_train, y_train)

[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 3.1859396532851685
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 3.1859396532851685
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 4.162790938052734
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 4.162790938052734
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 4.162790938052734
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 7.309341851257409
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alp

[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 0.7650364591655299
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 5.943283602003962
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 5.943283602003962
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 5.943283602003962
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 5.76703481839131
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 5.76703481839131
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha=

[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 2.2885273773919117
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 2.2885273773919117
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 2.2885273773919117
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 0.1517731740748361
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 0.1517731740748361
[LightGBM] [Warning] reg_alpha is set=0.0, reg_alpha= will be ignored. Current value: reg_alpha=0.0
[LightGBM] [Warning] Unknown parameter: 0.1517731740748361
[LightGBM] [Warning] reg_alpha is set=0.0, reg

GPOptimizer(estimator=LGBMClassifier(boosting_type='dart',
                                     class_weight='balanced',
                                     learning_rate=0.06619829018245553,
                                     max_depth=6, min_child_samples=21,
                                     n_estimators=490, num_leaves=48,
                                     random_state=42,
                                     reg_alpha =1.0658688513690675),
            num_runs=40, num_samples_per_run=10,
            param_distributions={'boosting_type': <distributions.CategoricalDistribution object at 0x0000...
                                 'min_child_samples': <distributions.IntUniformDistribution object at 0x000001E2DD08A7D0>,
                                 'n_estimators': <distributions.IntUniformDistribution object at 0x000001E2DD02A140>,
                                 'num_leaves': <distributions.IntUniformDistribution object at 0x000001E2DD08BA90>,
                           

In [112]:
gp_search.scores_history

array([-1.38244446, -1.24234261, -1.17510703, -1.08519698, -1.08328073,
       -1.57427871, -1.58651711, -1.26622241, -1.08865206, -1.08410907,
       -1.14252219, -1.12765036, -1.17971472, -1.22910788, -1.36632201,
       -1.21624848, -1.12862146, -1.52584761, -1.25064586, -1.32613662,
       -1.0724979 , -1.69946445, -1.16144468, -1.27767098, -1.28638445,
       -1.45272669, -1.18828677, -1.21189726, -1.09621166, -1.10712562,
       -1.27862943, -1.14153632, -1.1275754 , -1.2000032 , -1.12822314,
       -1.09666967, -1.08321278, -1.35660756, -1.12348419, -1.02969381])

In [116]:
gp_search.params_history['n_estimators']

array([485., 774., 389., 197., 332., 174., 661., 148., 414., 725., 734.,
       244.,  88., 274.,  72., 509., 525., 379., 659., 379., 788., 937.,
       730.,  82., 255., 272., 996., 935., 850., 353., 633., 588., 810.,
       336., 514., 369., 409.,  69., 886., 490.])

In [ ]:
tpe_search = TPEOptimizer(est, params_distr, num_runs=40, random_state=42, 
                        num_dry_runs=5, num_samples_per_run=10, 
                        scoring=log_loss)
tpe_search.fit(X_train, y_train)

Теперь попробуем визуализировать, что у нас получилось. Сделайте 4 сабплота, по одному на каждую стратегию поиска. На каждом графике изобразите лучшее найденное значение метрики, усредненное по запускам алгоритма, против номера итерации. Также нарисуйте область ± одного стандартного отклонения лучшего значения метрики. Кроме того, на том же графике сделайте scatter-plot **(номер итерации, значение метрики)** для этого запуска (именно для текущего набора гиперпараметра с итерации, а не лучшее, как для прошлого графика). Можете рисовать scatter-plot для всех повторных запусков сразу.

Прокомментируйте наблюдаемое. Какой(-ие) алгоритм(-ы) показали себя лучше всего? Какой(-ие) можно назвать более стабильным(-и)?

In [119]:
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(2,2, figsize = (20,20))

for ax in axes.flatten():
    ax.set_title("RandomSearchOptimizer's best scores (scaled)")
    ax.plot(np.arange(1,6),test_scores['RS']/5)
    ax.set_xlabel('Iterations')
    ax.set_ylabel('Score')
    
    for i in range(1, 4):
        ax.scatter(x=np.array([i]*len(scores_history['RS'][i])), 
                   y=scores_history['RS'][i-1]/5, color = 'r')
    ax.fill_between(np.arange(1, 4), 
                    (test_scores['RS']/3 - test_scores_sigma['RS']), 
                    (test_scores['RS']/3 + test_scores_sigma['RS']), color='b', alpha=.1)



axes[1,0].set_title("GPOptimizer's best scores (scaled)")
axes[1,0].plot(np.arange(1,6),test_scores['GP']/5)
axes[1,0].set_xlabel('Number of iter')
axes[1,0].set_ylabel('neg log loss')

axes[1,0].fill_between(np.arange(1,6), (test_scores['GP']/5-test_scores_sigma['GP']), 
                       (test_scores['GP']/5+test_scores_sigma['GP']), color='g', alpha=.2)

axes[0,1].set_title("TPEOptimizer's best scores (scaled)")
axes[0,1].plot(np.arange(1,6),test_scores['TPE']/5)
axes[0,1].set_xlabel('Number of iter')
axes[0,1].set_ylabel('neg log loss')

axes[0,1].fill_between(np.arange(1,6), (test_scores['TPE']/5-test_scores_sigma['TPE']), 
                       (test_scores['TPE']/5+test_scores_sigma['TPE']), color='g', alpha=.2)


axes[1,1].set_title("OptunaSearchOptimizerCV's best scores (scaled)")
axes[1,1].plot(np.arange(1,6),test_scores['Optuna']/5)
axes[1,1].set_xlabel('Number of iter')
axes[1,1].set_ylabel('neg log loss')

axes[1,1].fill_between(np.arange(1,6), (test_scores['Optuna']/5-test_scores_sigma['Optuna']), 
                       (test_scores['Optuna']/5+test_scores_sigma['Optuna']), color='g', alpha=.2)
plt.show()

**Задание 2.3 (1.5 балла).** Теперь посмотрим на распределение $p(y|x)$, которое получилось у `GPOptimizer`. Рассмотрим плоскость значений гиперпараметров числа деревьев и learning rate. Настройте ГП на выборку истории значений этих двух гиперпараметров одного из `GPOptimizer`, обученных вами выше. Ядро возьмите такое же, какое использовалось при поиске. Визуализируйте среднее, которое предсказывает ГП на плоскости (вам поможет `plt.imshow` или `plt.scatter`). Также добавьте на рисунок точки из истории поиска. Не забудьте про colorbar'ы и правильные подписи отметок на осях. Сделайте выводы. 

In [ ]:
# Your code here (⊃｡•́‿•̀｡)⊃

**Задание 2.4 (1.5 балла)**. Аналогичным образом визуализируйте значения $\log g(x)$ и $\log l(x)$ на той же плоскости для `TPEOptimizer`. Параметр `bandwidth` оценивайте так же, как и в алгоритме поиска. Не забудьте добавить на рисунок точки из истории поиска (каждую из двух групп поместите на свой график). Что можно сказать о получившихся распределениях?

In [ ]:
# Your code here (⊃｡•́‿•̀｡)⊃

**Задание 2.5 (0.5 балла)**. И на последок визуализируем, как изменялись распределения для категориального гиперпараметра у разных стратегий поиска. В этом пункте вам предлагается построить три рисунка:

1. Ддя `GPOptimizer`: постройте графики $\mu(c) \pm \sigma(c)$ для каждой категории $c$ против номера итерации (все на одном рисунке). 


2. Для `TPEOptimizer`: постройте area plot для значений $g(c)$ против номера итерации.


3. Аналогично пункту 2 для значений $l(c)$.

Прокомментируйте то, что получилось.

In [ ]:
# Your code here (⊃｡•́‿•̀｡)⊃